In [2]:
import pandas as pd
from pathlib import Path

import utils

In [4]:
df = pd.read_csv('urls_old.csv')

In [ ]:
for i, row in df.iterrows():  # downloading
    print(i)
    full_path = Path(f'audio_full/{utils.extract_youtube_code(row.url)}.wav')
    trim_path = Path(f'audio/{utils.extract_youtube_code(row.url)}.wav')

    if not full_path.is_file() and not trim_path.is_file():
        utils.download_youtube_audio(row.url, full_path, keep_video=False)

In [ ]:
for i, row in df.iterrows():  # trimming
    print(i)
    full_path = Path(f'audio_full/{utils.extract_youtube_code(row.url)}.wav')
    trim_path = Path(f'audio/{utils.extract_youtube_code(row.url)}.wav')

    if full_path.is_file() and not trim_path.is_file():
        print(full_path)
        utils.Audio.load(full_path).trim(utils.parse_time(row.start), utils.parse_time(row.end) + 30).save(trim_path)
        full_path.unlink()

In [26]:
import librosa
from tqdm.auto import tqdm

data = []

for i, row in tqdm(list(df.iterrows())):
    name = row.url.split('=')[-1]
    srt_path = Path(f'data_old/{name}.srt')
    audio_path = Path(f'audio/{name}.wav')
    if srt_path.is_file():
        if not audio_path.is_file():
            print(audio_path, 'does not exist')
            continue
        waveform, samling_rate = librosa.load(audio_path)
        data.append({
            'video_url': row.url,
            'video_segment_start_sec': row.start,
            'video_segment_end_sec': row.end + 30,
            'srt_transcription': srt_path.read_text(),
            'audio': {
                'array': waveform,
                'samling_rate': samling_rate,
            }
        })

100%|██████████| 48/48 [00:00<00:00, 155.64it/s]

audio/cEz8pMOnQAs.wav does not exist


In [31]:
from datasets import Dataset

dataset = Dataset.from_list(data)
dataset

Dataset({
    features: ['video_url', 'video_segment_start_sec', 'video_segment_end_sec', 'srt_transcription', 'audio'],
    num_rows: 17
})

In [33]:
dataset.save_to_disk('multivariant_v1')

Saving the dataset (0/1 shards):   0%|          | 0/17 [00:00<?, ? examples/s]

Saving the dataset (1/1 shards): 100%|██████████| 17/17 [00:00<00:00, 55.25 examples/s]


In [ ]:
from datasets import load_from_disk

dataset = load_from_disk('multivariant_v1')
print(dataset)

Dataset({
    features: ['video_url', 'video_segment_start_sec', 'video_segment_end_sec', 'srt_transcription', 'audio'],
    num_rows: 17
})
